<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/AnalysingPretrainedLongFormerModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Performance of Longformer on SQAUD2

In [1]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 1.8MB 16.1MB/s 
     |████████████████████████████████| 890kB 51.8MB/s 
     |████████████████████████████████| 3.2MB 53.1MB/s 
     |████████████████████████████████| 184kB 16.0MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 245kB 60.7MB/s 


In [38]:
from transformers import squad_convert_examples_to_features
from transformers.data.processors.squad import SquadV2Processor, SquadFeatures
from tqdm.notebook import tqdm
import torch
device =  'cuda' if torch.cuda.is_available() else 'cpu'

In [39]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("mrm8488/longformer-base-4096-finetuned-squadv2", use_fast=False)
model = AutoModelForQuestionAnswering.from_pretrained("mrm8488/longformer-base-4096-finetuned-squadv2")
model.to(device)

In [36]:
from datasets import load_dataset, load_metric
datasets = load_dataset("squad_v2")

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/9cac55034b086140f0649ecb5c604d09d7da2f2f5b73a90caa2e2bcc1f5cac09)


In [75]:
valid_dataset = datasets['validation']
count=0
for example in tqdm(valid_dataset):
  print(example)
  count+=1
  if count==10:
    break

{'answers': {'answer_start': [159, 159, 159, 159], 'text': ['France', 'France', 'France', 'France']}, 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'id': '56ddde6b9a695914005b9628', 'question': 'In what country is Normandy located?', 'title': 'Normans'}
{'answers': {'answer_start': [94, 8

In [62]:
inputs = tokenizer(example['question'], example['context'], return_tensors="pt")
inputs.to(device)
output = model(**inputs)
start_index = torch.argmax(output['start_logits'])
end_index = torch.argmax(output['end_logits'])
start_index, end_index

(tensor(53, device='cuda:0'), tensor(53, device='cuda:0'))

In [66]:
ans_ids = inputs['input_ids'][0][start_index :end_index+1]
ans_ids

tensor([1470], device='cuda:0')

In [89]:
predictions=[]
references=[]
for example in tqdm(valid_dataset):
  inputs = tokenizer(example['question'], example['context'], return_tensors="pt")
  inputs.to(device)
  output = model(**inputs)
  start_index = torch.argmax(output['start_logits'])
  end_index = torch.argmax(output['end_logits'])
  ans_ids = inputs['input_ids'][0][start_index :end_index+1]
  answer = tokenizer.decode(ans_ids)
  answer = tokenizer.clean_up_tokenization(answer).strip()
  answer = '' if answer=='<s>' else answer
  no_answer_probability = 1 if len(example['answers']['answer_start'])!=0 else 0
  """
  predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22', 'no_answer_probability': 0.}]
  references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
  """
  pred = {'prediction_text':answer, 'id': example['id'], 'no_answer_probability' : no_answer_probability}
  ref = {'answers': example['answers'] , 'id': example['id']}
  predictions.append(pred)
  references.append(ref)

In [90]:
from datasets import load_metric

In [91]:
squad_metric = load_metric('squad_v2')
score = squad_metric.compute(predictions=predictions, references=references)
score

{'HasAns_exact': 77.64844804318489,
 'HasAns_f1': 84.55162253066118,
 'HasAns_total': 5928,
 'NoAns_exact': 81.27838519764508,
 'NoAns_f1': 81.27838519764508,
 'NoAns_total': 5945,
 'best_exact': 79.4660153288975,
 'best_exact_thresh': 1.0,
 'best_f1': 82.91266052065693,
 'best_f1_thresh': 1.0,
 'exact': 79.4660153288975,
 'f1': 82.91266052065696,
 'total': 11873}